In [46]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [47]:
# Chage cwd
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\MLB')
print(f'CWD: \n{os.getcwd()}')

CWD: 
c:\Users\tyler\OneDrive\Documents\Python\MLB


In [48]:
# Load schedules
schedule = pd.DataFrame()
for season in range(2022-5, 2023):
    schedule = schedule.append(pd.read_csv(f'backend/data/schedules/{season}.csv'), ignore_index=True)

schedule['date'] = pd.to_datetime(schedule['date'])
schedule = schedule.drop_duplicates(['date', 'visitor', 'home'])
schedule.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12995 entries, 0 to 13244
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     12995 non-null  datetime64[ns]
 1   visitor  12995 non-null  object        
 2   home     12995 non-null  object        
 3   season   12995 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 507.6+ KB


In [49]:
# Load scores
scores = pd.read_csv('backend/data/scores/boxscore.csv')
scores['date'] = pd.to_datetime(scores['date'])
scores = scores.drop_duplicates(['date', 'visitor', 'home', 'team'])

scores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22050 entries, 0 to 23015
Data columns (total 26 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     22050 non-null  datetime64[ns]
 1   visitor  22050 non-null  object        
 2   home     22050 non-null  object        
 3   team     22050 non-null  object        
 4   1        22050 non-null  int64         
 5   2        22050 non-null  int64         
 6   3        22050 non-null  int64         
 7   4        22050 non-null  int64         
 8   5        22050 non-null  object        
 9   6        22042 non-null  object        
 10  7        22026 non-null  object        
 11  8        21808 non-null  object        
 12  9        21771 non-null  object        
 13  R        22050 non-null  int64         
 14  H        22050 non-null  int64         
 15  E        22050 non-null  int64         
 16  10       1836 non-null   float64       
 17  11       837 non-null    float6

In [50]:
# Load batting totals
batting_totals = pd.read_csv('backend/data/batting/totals.csv')
batting_totals['date'] = pd.to_datetime(batting_totals['date'])
batting_totals['wpa-'] = batting_totals['wpa-'].apply(lambda x: float(x.strip("%")))
batting_totals['cwpa'] = batting_totals['cwpa'].apply(lambda x: float(x.strip("%")))
batting_totals = batting_totals.fillna(0)
batting_totals = batting_totals[batting_totals['ab'] != 0]
batting_totals = batting_totals.drop_duplicates(['date', 'visitor', 'home', 'team'])

batting_totals.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22050 entries, 0 to 23015
Data columns (total 46 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     22050 non-null  datetime64[ns]
 1   visitor  22050 non-null  object        
 2   home     22050 non-null  object        
 3   team     22050 non-null  object        
 4   ab       22050 non-null  int64         
 5   r        22050 non-null  int64         
 6   h        22050 non-null  int64         
 7   rbi      22050 non-null  int64         
 8   bb       22050 non-null  int64         
 9   so       22050 non-null  int64         
 10  pa       22050 non-null  int64         
 11  ba       22050 non-null  float64       
 12  obp      22050 non-null  float64       
 13  slg      22050 non-null  float64       
 14  ops      22050 non-null  float64       
 15  pit      22050 non-null  int64         
 16  str      22050 non-null  int64         
 17  wpa      22050 non-null  float6

In [51]:
# Load pitching totals
pitching_totals = pd.read_csv('backend/data/pitching/totals.csv')
pitching_totals['date'] = pd.to_datetime(pitching_totals['date'])
pitching_totals['cwpa'] = pitching_totals['cwpa'].apply(lambda x: float(x.strip('%')))
pitching_totals = pitching_totals.drop_duplicates(['date', 'visitor', 'home', 'team'])

pitching_totals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22050 entries, 0 to 23015
Data columns (total 30 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     22050 non-null  datetime64[ns]
 1   visitor  22050 non-null  object        
 2   home     22050 non-null  object        
 3   team     22050 non-null  object        
 4   ip       22050 non-null  float64       
 5   h        22050 non-null  int64         
 6   r        22050 non-null  int64         
 7   er       22050 non-null  int64         
 8   bb       22050 non-null  int64         
 9   so       22050 non-null  int64         
 10  hr       22050 non-null  int64         
 11  era      22050 non-null  float64       
 12  bf       22050 non-null  int64         
 13  pit      22050 non-null  int64         
 14  str      22050 non-null  int64         
 15  ctct     22050 non-null  int64         
 16  sts      22050 non-null  int64         
 17  stl      22050 non-null  int64 

In [52]:
# Load pitching details
pitching_details = pd.read_csv('backend/data/pitching/details.csv')
pitching_details['date'] = pd.to_datetime(pitching_details['date'])
pitching_details['cwpa'] = pitching_details['cwpa'].apply(lambda x: float(x.strip('%')))

pitching_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101027 entries, 0 to 101026
Data columns (total 31 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   date     101027 non-null  datetime64[ns]
 1   visitor  101027 non-null  object        
 2   home     101027 non-null  object        
 3   team     101027 non-null  object        
 4   player   101027 non-null  object        
 5   ip       101027 non-null  float64       
 6   h        101027 non-null  int64         
 7   r        101027 non-null  int64         
 8   er       101027 non-null  int64         
 9   bb       101027 non-null  int64         
 10  so       101027 non-null  int64         
 11  hr       101027 non-null  int64         
 12  era      100988 non-null  float64       
 13  bf       101027 non-null  int64         
 14  pit      101001 non-null  float64       
 15  str      101001 non-null  float64       
 16  ctct     100922 non-null  float64       
 17  sts      1

In [53]:
starters = pd.read_csv('backend/data/pitching/starters.csv')
starters['date'] = pd.to_datetime(starters['date'])
bullpen = pd.read_csv('backend/data/pitching/bullpen.csv')
bullpen['date'] = pd.to_datetime(bullpen['date'])

for team in pitching_details['team'].unique():
    print(team)
    team_schedule = pitching_details[
        (pitching_details['team'] == team) &
        (pitching_details['date'] > starters['date'].max()) 
    ]
    for date in team_schedule['date'].unique():
        game = team_schedule[team_schedule['date'] == date].copy()
        starters = starters.append(game.iloc[0, :], ignore_index=True)
        bullpen = bullpen.append(game.iloc[1:, :], ignore_index=True)



San Francisco Giants
Arizona Diamondbacks
Chicago Cubs
St. Louis Cardinals
New York Yankees
Tampa Bay Rays
Toronto Blue Jays
Baltimore Orioles
Pittsburgh Pirates
Boston Red Sox
Philadelphia Phillies
Cincinnati Reds
Seattle Mariners
Houston Astros
San Diego Padres
Los Angeles Dodgers
Colorado Rockies
Milwaukee Brewers
Kansas City Royals
Minnesota Twins
Atlanta Braves
New York Mets
Los Angeles Angels
Oakland Athletics
Cleveland Indians
Texas Rangers
Miami Marlins
Washington Nationals
Detroit Tigers
Chicago White Sox
Cleveland Guardians


In [54]:
starters.drop_duplicates(
    ['date', 'visitor', 'home', 'team']
).to_csv('backend/data/pitching/starters.csv', index=False)


bullpen.groupby(
    ['date', 'visitor', 'home', 'team']
).sum(
).reset_index(
).drop_duplicates(
    ['date', 'visitor', 'home', 'team']
).to_csv('backend/data/pitching/bullpen.csv', index=False)

In [55]:
starters = pd.read_csv('backend/data/pitching/starters.csv')
starters['date'] = pd.to_datetime(starters['date'])
starters = starters.drop_duplicates(['date', 'visitor', 'home', 'team'])
starters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21941 entries, 0 to 21940
Data columns (total 31 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   acli     21941 non-null  float64       
 1   ali      21941 non-null  float64       
 2   bb       21941 non-null  float64       
 3   bf       21941 non-null  float64       
 4   ctct     21934 non-null  float64       
 5   cwpa     21941 non-null  float64       
 6   date     21941 non-null  datetime64[ns]
 7   er       21941 non-null  float64       
 8   era      21940 non-null  float64       
 9   fb       21936 non-null  float64       
 10  gb       21936 non-null  float64       
 11  gsc      21936 non-null  float64       
 12  h        21941 non-null  float64       
 13  home     21941 non-null  object        
 14  hr       21941 non-null  float64       
 15  ip       21941 non-null  float64       
 16  ir       5 non-null      float64       
 17  is       5 non-null      float6

In [56]:
bullpen = pd.read_csv('backend/data/pitching/bullpen.csv')
bullpen['date'] = pd.to_datetime(bullpen['date'])
bullpen = bullpen.drop_duplicates(['date', 'visitor', 'home', 'team'])
bullpen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21746 entries, 0 to 21745
Data columns (total 30 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     21746 non-null  datetime64[ns]
 1   visitor  21746 non-null  object        
 2   home     21746 non-null  object        
 3   team     21746 non-null  object        
 4   ip       21746 non-null  float64       
 5   h        21746 non-null  int64         
 6   r        21746 non-null  int64         
 7   er       21746 non-null  int64         
 8   bb       21746 non-null  int64         
 9   so       21746 non-null  int64         
 10  hr       21746 non-null  int64         
 11  era      21746 non-null  float64       
 12  bf       21746 non-null  int64         
 13  pit      21746 non-null  float64       
 14  str      21746 non-null  float64       
 15  ctct     21746 non-null  float64       
 16  sts      21746 non-null  float64       
 17  stl      21746 non-null  float6

In [97]:
def feature_engineer(df):
    # Function for feature engineering stats for batting
    df[f'ba'] = df[f'h'] / df[f'ab']
    df[f'obp'] = (df[f'h'] + df[f'bb'] + df[f'hbp']) / (df[f'ab'] + df[f'bb'] + df[f'hbp'] + df[f'sf'])
    df[f'1b'] = df[f'h'] - (df[f'2b'] + df[f'3b'] + df[f'hr'])
    df[f'slg'] = (df[f'1b'] + 2*df[f'2b'] + 3*df[f'3b'] + 4*df[f'hr']) / df[f'ab']
    df[f'ops'] = df[f'obp'] + df[f'slg']

    return df

In [98]:
batting_totals['h_target'] = batting_totals['h']
batting_totals['season'] = batting_totals['date'].dt.year

df = batting_totals.set_index(
    ['team']
).sort_index()

batting_season = pd.DataFrame()
for index in df.index.unique():
    team_df = df.loc[index, :].sort_values(
        ['date'], 
        ascending=True
    ).copy()

    for col in list(set(df.columns).difference(['date', 'visitor', 'home', 'opponent', 'h_target'])):
        # team_df[col] = team_df[col].rolling(41, closed='left').mean()
        team_df[col] = team_df[col].ewm(span=5, min_periods=5).mean().shift(1)

    team_df = feature_engineer(team_df)
    
    batting_season = batting_season.append(team_df.reset_index(), ignore_index=True)

batting_season = batting_season.dropna(axis=0)
batting_season.shape

(21895, 50)

In [99]:
corr = {'col': [], 'corr': []}
for col in list(set(batting_season.columns).difference(['date', 'visitor', 'home', 'team', 'opponent', 'h_target'])):
    corr['col'].append(col)
    corr['corr'].append(batting_season[col].corr(batting_season['h_target']))

corr = pd.DataFrame(corr)
corr.sort_values(['corr'], ascending=False).head(10)

,col,corr
37,ops,0.071579
42,slg,0.070779
33,h,0.063878
40,ba,0.062345
39,2b,0.061958
18,hr,0.053610
31,obp,0.051636
11,ab,0.046274
43,gdp,0.045668
2,rbi,0.043502


In [60]:
batting_totals['h_target'] = batting_totals['h']
batting_totals['season'] = batting_totals['date'].dt.year
batting_totals['opponent'] = np.where(batting_totals['team'] == batting_totals['home'], batting_totals['home'], batting_totals['visitor'])


df = batting_totals.set_index(
    ['team', 'opponent']
).sort_index()

batting_team = pd.DataFrame()
for index in df.index.unique():
    team_df = df.loc[index, :].sort_values(
        ['date'], 
        ascending=True
    ).copy()

    for col in list(set(df.columns).difference(['date', 'visitor', 'home', 'h_target'])):
        # team_df[col] = team_df[col].rolling(15, closed='left', min_periods=15).mean()
        team_df[col] = team_df[col].ewm(span=20, min_periods=20).mean().shift(1)

    team_df = feature_engineer(team_df)
    
    batting_team = batting_team.append(team_df.reset_index(), ignore_index=True)

batting_team = batting_team.dropna(axis=0)
batting_team.shape

(21430, 50)

In [61]:
corr = {'col': [], 'corr': []}
for col in list(set(batting_team.columns).difference(['date', 'visitor', 'home', 'team', 'opponent', 'h_target'])):
    corr['col'].append(col)
    corr['corr'].append(batting_team[col].corr(batting_team['h_target']))

corr = pd.DataFrame(corr)
corr.sort_values(['corr'], ascending=False).head(10)

,col,corr
33,h,0.081916
40,ba,0.079464
37,ops,0.077485
42,slg,0.073740
39,2b,0.066494
11,ab,0.065704
31,obp,0.062289
18,hr,0.058741
43,gdp,0.054506
2,rbi,0.054486


In [62]:
def feature_engineer(df):
    # Function for feature engineering stats for pitchers
    df[f'era'] = 9 * df[f'er'] / df[f'ip']

    return df

In [63]:
df = pd.merge(
    bullpen, 
    pitching_totals[['date', 'visitor', 'home', 'team', 'h']],
    left_on=['date', 'visitor', 'home', 'team'],
    right_on=['date', 'visitor', 'home', 'team'],
    suffixes=('', '_target')
)
df['season'] = df['date'].dt.year

df = df.set_index(
    ['team', 'season']
).sort_index()

bullpen_season = pd.DataFrame()
for index in df.index.unique():
    team_df = df.loc[index, :].sort_values(
        ['date'], 
        ascending=True
    ).copy()

    for col in list(set(df.columns).difference(['date', 'visitor', 'home', 'h_target', 'player'])):
        team_df[col] = team_df[col].rolling(50, closed='left', min_periods=5).mean()
        # team_df[col] = team_df[col].ewm(span=30, min_periods=2).mean().shift(1)

    team_df = feature_engineer(team_df)
    
    bullpen_season = bullpen_season.append(team_df.reset_index(), ignore_index=True)

bullpen_season = bullpen_season.dropna(axis=0)
bullpen_season.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20846 entries, 5 to 21745
Data columns (total 32 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   team      20846 non-null  object        
 1   season    20846 non-null  int64         
 2   date      20846 non-null  datetime64[ns]
 3   visitor   20846 non-null  object        
 4   home      20846 non-null  object        
 5   ip        20846 non-null  float64       
 6   h         20846 non-null  float64       
 7   r         20846 non-null  float64       
 8   er        20846 non-null  float64       
 9   bb        20846 non-null  float64       
 10  so        20846 non-null  float64       
 11  hr        20846 non-null  float64       
 12  era       20846 non-null  float64       
 13  bf        20846 non-null  float64       
 14  pit       20846 non-null  float64       
 15  str       20846 non-null  float64       
 16  ctct      20846 non-null  float64       
 17  sts       20

In [64]:
corr = {'col': [], 'corr': []}
for col in list(set(bullpen_season.columns).difference(['date', 'visitor', 'home', 'team', 'h_target', 'player'])):
    corr['col'].append(col)
    corr['corr'].append(bullpen_season[col].corr(bullpen_season['h_target']))

corr = pd.DataFrame(corr)
corr.sort_values(['corr'], ascending=False).head(10)

,col,corr
6,era,0.059179
23,h,0.046631
15,er,0.045449
4,r,0.039399
12,hr,0.031072
11,is,0.028859
16,gb,0.028071
26,ld,0.027590
0,ir,0.024609
1,unk,0.019405


In [65]:
df = pd.merge(
    bullpen, 
    pitching_totals[['date', 'visitor', 'home', 'team', 'h']],
    left_on=['date', 'visitor', 'home', 'team'],
    right_on=['date', 'visitor', 'home', 'team'],
    suffixes=('', '_target')
)
df['season'] = df['date'].dt.year
df['opponent'] = np.where(df['team'] == df['home'], df['home'], df['visitor'])

df = df.set_index(
    ['team', 'opponent']
).sort_index()

bullpen_team = pd.DataFrame()
for index in df.index.unique():
    team_df = df.loc[index, :].sort_values(
        ['date'], 
        ascending=True
    ).copy()

    for col in list(set(df.columns).difference(['date', 'visitor', 'home', 'h_target', 'player'])):
        team_df[col] = team_df[col].rolling(60, closed='left', min_periods=30).mean()
        # team_df[col] = team_df[col].ewm(span=30, min_periods=2).mean().shift(1)

    team_df = feature_engineer(team_df)
    
    bullpen_team = bullpen_team.append(team_df.reset_index(), ignore_index=True)

bullpen_team = bullpen_team.dropna(axis=0)
bullpen_team.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20824 entries, 30 to 21745
Data columns (total 33 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   team      20824 non-null  object        
 1   opponent  20824 non-null  object        
 2   date      20824 non-null  datetime64[ns]
 3   visitor   20824 non-null  object        
 4   home      20824 non-null  object        
 5   ip        20824 non-null  float64       
 6   h         20824 non-null  float64       
 7   r         20824 non-null  float64       
 8   er        20824 non-null  float64       
 9   bb        20824 non-null  float64       
 10  so        20824 non-null  float64       
 11  hr        20824 non-null  float64       
 12  era       20824 non-null  float64       
 13  bf        20824 non-null  float64       
 14  pit       20824 non-null  float64       
 15  str       20824 non-null  float64       
 16  ctct      20824 non-null  float64       
 17  sts       2

In [66]:
corr = {'col': [], 'corr': []}
for col in list(set(bullpen_team.columns).difference(['date', 'visitor', 'home', 'team', 'opponent', 'h_target', 'player'])):
    corr['col'].append(col)
    corr['corr'].append(bullpen_team[col].corr(bullpen_team['h_target']))

corr = pd.DataFrame(corr)
corr.sort_values(['corr'], ascending=False).head(10)

,col,corr
6,era,0.065282
15,er,0.042358
23,h,0.042132
4,r,0.037933
1,unk,0.029776
16,gb,0.026986
12,hr,0.024319
26,ld,0.020644
11,is,0.017466
20,fb,0.011584


In [67]:
starters['season'] = starters['date'].dt.year
starters['player'] = starters['player'].apply(lambda x: x.strip(','))

df = pd.merge(
    starters, 
    pitching_totals[['date', 'visitor', 'home', 'team', 'h']],
    left_on=['date', 'visitor', 'home', 'team'],
    right_on=['date', 'visitor', 'home', 'team'],
    suffixes=('', '_target')
)

df = df.set_index(
    ['player']
).sort_index(
).drop(
    ['is', 'ir'], 
    axis=1
)

starters_season_sma = pd.DataFrame()
for index in df.index.unique():
    team_df = df.loc[index, :].copy()
    if len(pd.DataFrame().append(team_df, ignore_index=True)) <= 1:
        pass
    else:
        team_df = team_df.sort_values(by=['date'])
        for col in list(set(df.columns).difference(['date', 'visitor', 'home', 'team', 'h_target', 'player', 'is', 'ir', 'opponent'])):
            # team_df[col] = team_df[col].rolling(10, closed='left', min_periods=3).mean()
            team_df[col] = team_df[col].ewm(span=15, min_periods=1).mean().shift(1)

        team_df = feature_engineer(team_df)
        
        starters_season_sma = starters_season_sma.append(team_df.reset_index(), ignore_index=True)

starters_season_sma = starters_season_sma.dropna(axis=0)
starters_season_sma.shape

(21174, 31)

In [68]:
corr = {'col': [], 'corr': []}
for col in list(set(starters_season_sma.columns).difference(['date', 'visitor', 'home', 'team', 'h_target', 'player', 'opponent'])):
    corr['col'].append(col)
    corr['corr'].append(starters_season_sma[col].corr(starters_season_sma['h_target']))

corr = pd.DataFrame(corr)
corr.sort_values(['corr'], ascending=False).head(10)

,col,corr
19,h,0.084487
22,ld,0.073297
15,fb,0.072224
5,er,0.068770
8,r,0.067634
11,gb,0.057193
10,ctct,0.035281
0,hr,0.031488
12,era,0.018106
14,bf,0.014605


In [39]:
starters['season'] = starters['date'].dt.year
starters['player'] = starters['player'].apply(lambda x: x.strip(','))
starters['opponent'] = np.where(starters['home'] == starters['team'], starters['home'], starters['visitor'])

df = pd.merge(
    starters, 
    pitching_totals[['date', 'visitor', 'home', 'team', 'h']],
    left_on=['date', 'visitor', 'home', 'team'],
    right_on=['date', 'visitor', 'home', 'team'],
    suffixes=('', '_target')
)

df = df.set_index(
    ['player', 'opponent']
).sort_index(
).drop(
    ['is', 'ir'], 
    axis=1
)

starters_team_sma = pd.DataFrame()
for index in df.index.unique():
    team_df = df.loc[index, :].copy()

    if len(pd.DataFrame().append(team_df, ignore_index=True)) <= 1:
        pass
    else:
        team_df = team_df.sort_values(by=['date'])
        for col in list(set(df.columns).difference(['date', 'visitor', 'home', 'team', 'h_target', 'player', 'is', 'ir'])):
            # team_df[col] = team_df[col].rolling(5, closed='left', min_periods=2).mean()
            team_df[col] = team_df[col].ewm(span=5, min_periods=5).mean().shift(1)

        team_df = feature_engineer(team_df)
        
        starters_team_sma = starters_team_sma.append(team_df.reset_index(), ignore_index=True)

starters_team_sma = starters_team_sma.dropna(axis=0)
starters_team_sma.shape

(17574, 32)

In [40]:
corr = {'col': [], 'corr': []}
for col in list(set(starters_team_sma.columns).difference(['date', 'visitor', 'home', 'team', 'opponent', 'h_target', 'player'])):
    corr['col'].append(col)
    corr['corr'].append(starters_team_sma[col].corr(starters_team_sma['h_target']))

corr = pd.DataFrame(corr)
corr.sort_values(['corr'], ascending=False).head(10)

,col,corr
21,h,0.089732
18,fb,0.078531
3,r,0.073383
24,ld,0.073288
13,er,0.071771
5,era,0.063239
16,gb,0.055770
4,ctct,0.043520
10,hr,0.042257
6,bf,0.015140


In [112]:
batting_cols = ['date', 'visitor', 'home', 'team', 'h', 'h_target']
bullpen_cols = ['date', 'visitor', 'home', 'team', 'h']
starters_cols = ['date', 'visitor', 'home', 'team', 'h']

bullpen = pd.merge(
    bullpen_season[bullpen_cols], 
    bullpen_team[bullpen_cols],
    left_on=['date', 'visitor', 'home', 'team'],
    right_on=['date', 'visitor', 'home', 'team'],
    suffixes=('_bullpen_season', '_bullpen_team')
)

batting = pd.merge(
    batting_season[batting_cols],
    batting_team[batting_cols[:-1]],
    left_on=['date', 'visitor', 'home', 'team'],
    right_on=['date', 'visitor', 'home', 'team'],
    suffixes=('_batting_season', '_batting_team')
)

starters = pd.merge(
    starters_season_sma[starters_cols],
    starters_team_sma[starters_cols],
    left_on=['date', 'visitor', 'home', 'team'],
    right_on=['date', 'visitor', 'home', 'team'],
    suffixes=('_starters_season', '_starters_team')
)

batting_bullpen = pd.merge(
    batting,
    bullpen,
    left_on=['date', 'visitor', 'home', 'team'],
    right_on=['date', 'visitor', 'home', 'team'],
    suffixes=('_batting', '_bullpen')
)

df = pd.merge(
    batting_bullpen,
    starters,
    left_on=['date', 'visitor', 'home', 'team'],
    right_on=['date', 'visitor', 'home', 'team']
)

home_df = df[df['home'] == df['team']].copy()
visitor_df = df[df['visitor'] == df['team']].copy()

df = pd.merge(
    home_df, 
    visitor_df,
    left_on=['date', 'visitor', 'home'],
    right_on=['date', 'visitor', 'home'],
    suffixes=('_home', '_visitor')
)

df['most_hits'] = np.where(df['h_target_home'] > df['h_target_visitor'], 1, 0)
df['total_hits'] = df['h_target_home'] + df['h_target_visitor']

df = df.drop(['team_home', 'team_visitor', 'h_target_home', 'h_target_visitor'], axis=1)
df.to_csv('backend/preprocess/preprocess.csv', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6921 entries, 0 to 6920
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       6921 non-null   datetime64[ns]
 1   visitor                    6921 non-null   object        
 2   home                       6921 non-null   object        
 3   h_batting_season_home      6921 non-null   float64       
 4   h_batting_team_home        6921 non-null   float64       
 5   h_bullpen_season_home      6921 non-null   float64       
 6   h_bullpen_team_home        6921 non-null   float64       
 7   h_starters_season_home     6921 non-null   float64       
 8   h_starters_team_home       6921 non-null   float64       
 9   h_batting_season_visitor   6921 non-null   float64       
 10  h_batting_team_visitor     6921 non-null   float64       
 11  h_bullpen_season_visitor   6921 non-null   float64       
 12  h_bull